In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("bci05.csv")
data.head()

,tag,sp,gx,gy,dbh,pom,date,codes,status
0,105951,ACACME,610.0,104.7,119.0,1,8924.0,M,A
1,132160,ACACME,534.8,241.3,116.0,1,8922.0,*,A
2,132234,ACACME,539.4,242.3,NaN,0,8922.0,DN,D
3,132235,ACACME,538.8,242.5,NaN,0,8922.0,DN,D
4,191542,ACACME,282.7,177.5,75.0,1,8825.0,*,A


In [3]:
data = data[data["status"]=='A'][["tag", "sp", "gx", "gy"]]

In [4]:
# 1) Species spotting
types = data['sp'].value_counts().keys()
S = len(types)
print("There are {} different species".format(S))

There are 299 different species


In [5]:
# 2) Subsampling
# 50 * 50 meters
data["i"], data["j"] = data["gx"]//50, data["gy"]//50
# a)
cell_pop = data.groupby(["i", "j"])["sp"].value_counts()
# b) shaping them in a matrix
cell_pop_M = cell_pop.unstack().stack(dropna=False).fillna(0).astype(int)
cell_pop_M = np.array(cell_pop_M).reshape(20,10, 299) # i, j, specie

# this prints are just for understanding how to work with this dataset
#print("Presence and aboundance in subplot (0,0) : \n", cell_pop_M[0,0,:], '\n')

#print("Aboundances of the various species: \n", cell_pop_M.sum(axis=(0,1)), '\n')

presence = np.count_nonzero(cell_pop_M, axis = (0,1))
#print("Absolute presence for each species: \n", presence, '\n')

p_i = presence/(200)
#print("Relative presence for each species: \n", p_i, '\n')

# S+ - S-
S_present = np.count_nonzero(cell_pop_M, axis = (2)).flatten()

S_absent = S - S_present

# In each subplot there are more absent species than present
S_pm = S_present - S_absent

# constraint C_0 = < (S+ - S-)^2 >
C_0 = np.sum(np.power(S_pm,2))


## 3 - Max Ent 1

Analytical derivation of the tuned Lagrangian multipliers as functions of the constraints.

$$ \lambda_i = -\frac{1}{2} \cdot ln(\frac{1+ m_i}{1 - m_i} )$$

In [6]:
m_i = 2*p_i - 1
eps = 10e-06 # a small regularization in order to avoid devergences
l_i = -0.5 * np.log((1 + m_i + eps)/(1 - m_i + eps) )

## 4 - Max Ent 2

### Constraints for Max Ent 2

The constraints that we are going to use are:
* $m_i = <\sigma_i>_{model} = C_i(\vec{\sigma}) $ with coupled parameters $\lambda_i, i = 1,\ldots, S$
* $<(S_+ - S_-)^2>_{exp} = <(\sum_{j=1}^{S}\sigma_j)^2>_{model} = C_0(\vec{\sigma})$ with coupled parameter $\lambda_0 = K/S$

To initialize the Lagrange multipliers we have two possible choices: extracting them from a gaussian distribution centered in 0 or to take the initial $\lambda_i$ as the one of the previous point and for $K'$ using a gaussian with variance that is a funtion of S. [WORK IN PROGRESS]

### Gradient descent function

The objective function that we want to minimize is the Kullback–Leibler divergence $D_{KL}(P_{exp}/P_{model})$.

The derivatives of the KL-divergence w.r.t. the Lagrangian multipliers are:

$$ \frac{\partial D_{KL}}{\partial \lambda_a} = <C_a(\vec{\sigma})>_{model}-<C_a(\vec{\sigma})>_{exp} $$

More in concrete:

$$\frac{\partial D_{KL}(t)}{\partial \lambda_0} = <(\sum_{j=1}^{S}\sigma_j)^2>_{model(t)} - <(S_+ - S_-)^2>_{exp}  $$

$$\frac{\partial D_{KL}(t)}{\partial \lambda_i} = <\sigma_i>_{model(t)} - m_i $$


Thus the update rule for gradient descent will be:

$$\lambda_a(t+1) \leftarrow \lambda_a(t) - \eta \cdot \frac{\partial D_{KL}(t)}{\partial \lambda_a}$$


### Stopping criteria
Then we want to run the cycle of Metropolis and gradient descent until a stopping criteria is met.
Possible choices are:
* fixed number of iterations
* margin of improvement under a certain threshold

In [7]:
import metropolis as M

In [162]:
# define the constraint functions
# s is thought as a configuration of S spins

def K_constraint(s):
    return np.power(s.sum(),2)

#l_constraints = [lambda s : s[i] for i in range(S)]
l_constraints = []
# each lambda is the projection of the vector to its i-th coordinate
for i in range(S):
    l_constraints.append(lambda s, j=i: s[j])
    
constraints_funcs = [K_constraint] + l_constraints
print(len(constraints_funcs))

300


In [166]:
# initialize the lagrangian parameters
l_0 = np.random.randn()
lagrange_multipliers = np.concatenate((np.array([l_0]), l_i))

# the learning rate
eta = 0.1

# and the number of iterations
max_iter = 100

exp_constraints = np.concatenate((np.array([C_0]), m_i))
print(len(exp_constraints))
print(len(lagrange_multipliers))

300
300


In [173]:
def gradient(model_configs, constraints_funcs, exp_constraints):
    # we feed to each constraint function an array of arrays, made by replicas of configurations of S spins
    N = len(model_configs)
    C = len(exp_constraints)
    #print("Number of constraints: ", C)
    #print("Shape of simulated configurations: ", model_configs.shape)
    #print("Number of constraints functions: ", len(constraints_funcs))
    model_constraints = np.zeros((N,C))
    for i in range(N):
        model_constraints[i] = np.array(list(map(lambda f: f(model_configs[i]), constraints_funcs)))
    #print(model_constraints[0].shape)
    # mean over the columns (replicas)
    mean_model_c = model_constraints.mean(axis = 0)
    #print(mean_model_c.shape)
    
    # returns the gradient (points in the direction of maximal growth)
    return mean_model_c - exp_constraints

In [36]:
help(M)

Help on module metropolis:

NAME
    metropolis

CLASSES
    builtins.object
        Metropolis
    
    class Metropolis(builtins.object)
     |  Metropolis(S, M=100, N=1000, max_acceptance=0.1)
     |  
     |  Metropolis algorithm for an Ising model with the Hamiltonian given by maximum entropy principle.
     |  
     |  Parameters
     |  ------------
     |  
     |  S              : number of spins of a configuration (size of the system)
     |  M              : number of configurations used to compute the running rate of acceptance (memory)
     |  N              : number of configurations to be sampled
     |  max_acceptance : acceptance rate under which starts the importance sampling (es. 0.10)
     |  
     |  Attributes
     |  ------------
     |  
     |  L_multipliers    : list or numpy array of m Lagrange multipliers 
     |  constraint_funcs : list of m functions of the spins of a configuration, to implement the max-ent constraints
     |  energy           : the Hamilt

In [175]:
import importlib
importlib.reload(M)

<module 'metropolis' from '/home/nicola/Nicola_unipd/QuartoAnno/TODO/StatMech Complex Systems/Exercise/Ex1/Forests/metropolis.py'>

In [176]:
# instance of the class 
model = M.Metropolis(S, max_acceptance = 0.1)

In [178]:
from tqdm import tqdm_notebook, tnrange

for i in tnrange(max_iter):
    model = M.Metropolis(S, max_acceptance = 0.1)
    model.set_hamiltonian(lagrange_multipliers, constraints_funcs )
    model.sample()
    lagrange_multipliers = lagrange_multipliers - eta*gradient(model.configs, constraints_funcs, exp_constraints)

Acceptance rate criteria satisfied after 379 attempts.
Acceptance rate criteria satisfied after 209 attempts.
Acceptance rate criteria satisfied after 223 attempts.
Acceptance rate criteria satisfied after 112 attempts.
Acceptance rate criteria satisfied after 259 attempts.
Acceptance rate criteria satisfied after 334 attempts.
Acceptance rate criteria satisfied after 386 attempts.
Acceptance rate criteria satisfied after 234 attempts.
Acceptance rate criteria satisfied after 345 attempts.
Acceptance rate criteria satisfied after 494 attempts.
Acceptance rate criteria satisfied after 259 attempts.
Acceptance rate criteria satisfied after 117 attempts.
Acceptance rate criteria satisfied after 219 attempts.
Acceptance rate criteria satisfied after 372 attempts.
Acceptance rate criteria satisfied after 138 attempts.
Acceptance rate criteria satisfied after 156 attempts.
Acceptance rate criteria satisfied after 170 attempts.
Acceptance rate criteria satisfied after 335 attempts.
Acceptance